In [2]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
ACCOUNTS_WITH_BUNDLER_AND_DUNS = dataiku.Dataset("ACCOUNTS_WITH_BUNDLER_AND_DUNS")
ACCOUNTS_WITH_BUNDLER_AND_DUNS_df = ACCOUNTS_WITH_BUNDLER_AND_DUNS.get_dataframe()
df = ACCOUNTS_WITH_BUNDLER_AND_DUNS_df

In [6]:
df['DUNS'] = df['DUNS'].astype('Int64', errors='ignore')
df['DNB_DUNS_NUMBER'] = df['DNB_DUNS_NUMBER'].astype('Int64', errors='ignore')
df['DNB_GLOBAL_ULT_NUMBER'] = df['DNB_GLOBAL_ULT_NUMBER'].astype('Int64', errors='ignore')
df['DNB_GLOBAL_ULT_NUMBER'] = df['DNB_GLOBAL_ULT_NUMBER'].astype('Int64', errors='ignore')
df['DNB_STATE'] = 'Unknown'
df['EDW_STATE'] = 'Unknown'
df.head()

,CUSTOMER_ACCOUNT_ID,CUSTOMER_ACCOUNT_NAME,EDW_CUSTOMER_NAME,DUNS,IS_BUNDLER,DNB_GLOBAL_ULT_NUMBER,DNB_GLOBAL_ULT_NAME,DNB_DOMESTIC_ULT_NUMBER,DNB_DOMESTIC_ULTIMATE_NAME,DNB_HQ_NUMBER,DNB_HQ_NAME,DNB_DUNS_NUMBER,DNB_BUSINESS_NAME,DNB_STATE,EDW_STATE
0,6121005966569,HEIDLER ROOFING SERVICES INC,HEIDLER ROOFING SERVICES INC,152141495,False,152141495.0,"Heidler Roofing Services, Inc.",152141495.0,"Heidler Roofing Services, Inc.",152141495.0,"Heidler Roofing Services, Inc.",152141495,"Heidler Roofing Services, Inc.",Unknown,Unknown
1,9100001804959,HEIDLER ROOFING SERVICES INC,NaN,152141495,True,152141495.0,"Heidler Roofing Services, Inc.",152141495.0,"Heidler Roofing Services, Inc.",152141495.0,"Heidler Roofing Services, Inc.",152141495,"Heidler Roofing Services, Inc.",Unknown,Unknown
2,9100006532335,ANDERSON HOMES,NaN,608585766,True,608585766.0,Anderson Homes,608585766.0,Anderson Homes,608585766.0,Anderson Homes,608585766,Anderson Homes,Unknown,Unknown
3,9100008971857,ARARAT TRADING INC,NaN,255895117,True,NaN,NaN,NaN,NaN,NaN,NaN,255895117,Ararat Trading (2002) Corp.,Unknown,Unknown
4,9100005816790,ANGELS TAXI LLC,NaN,24832690,True,NaN,NaN,NaN,NaN,NaN,NaN,24832690,Angel's Taxi LLC,Unknown,Unknown


In [0]:
#ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS_df = 

# Write recipe outputs
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS = dataiku.Dataset("ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS")
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS.write_with_schema(ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS_df)